In [1]:
import networkx as nx
import torch
import numpy as np

import sys
sys.path.append("../codes")
from utils import no_growth_barabasi, no_preferential_attachment

In [10]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data, DataLoader

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_classes):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(1, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

In [11]:
def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


In [12]:
def make_data_from_nx(G, label):
    # G = nx.barabasi_albert_graph(100, 2)
    degs = list( dict(G.degree()).values())

    # node attribute => degree
    x = torch.tensor([[d] for d in degs], dtype=torch.float)

    # make edge
    edge_index = np.array(G.edges).T.tolist()
    edge_index = torch.tensor(edge_index, dtype=torch.long)

    return Data(x=x, edge_index=edge_index, y=torch.tensor([label]))

In [5]:
data_list = []
for _ in range(100):
    # BA
    G = nx.barabasi_albert_graph(100, 2)
    data = make_data_from_nx(G, 0)
    data_list.append(data)
    
    # no growth
    G = no_growth_barabasi(100, 200)
    data = make_data_from_nx(G, 1)
    data_list.append(data)
    
    # no attach
    G = no_preferential_attachment(100, 2)
    data = make_data_from_nx(G, 2)
    data_list.append(data)
    
    # random
    G = nx.gnp_random_graph(100, 0.04)
    data = make_data_from_nx(G, 3)
    data_list.append(data)

In [13]:
train_dataset = data_list[:300]
test_dataset = data_list[300:]

In [14]:
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [9]:
train_loader

In [15]:
model = GCN(hidden_channels=64, num_classes=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


for epoch in range(1, 10):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.5167, Test Acc: 0.5000
Epoch: 002, Train Acc: 0.6867, Test Acc: 0.7300
Epoch: 003, Train Acc: 0.7967, Test Acc: 0.7600
Epoch: 004, Train Acc: 0.7433, Test Acc: 0.7400
Epoch: 005, Train Acc: 0.7400, Test Acc: 0.7500
Epoch: 006, Train Acc: 0.6567, Test Acc: 0.7200
Epoch: 007, Train Acc: 0.8800, Test Acc: 0.9200
Epoch: 008, Train Acc: 0.9133, Test Acc: 0.9500
Epoch: 009, Train Acc: 0.8300, Test Acc: 0.8900
